# Reserve and configure FABRIC resources for "Setting Intermediate Network"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [3]:
routers_access_network = 3
number_of_AS=3
routers_path=2
routers_final_network=3
cdn='no'
redundancy=False

In [4]:
hops=3

In [18]:
slice_name="intermediate-network-8"
slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [19]:
import random


routers=[1]
if not redundancy:
    for i in range(hops-1):
        routers.append(1)
else:
    for i in range(hops-2):
        routers.append(random.randint(1,3))
    routers.append(1)
print(routers)
print(sum(routers))
sum_if=0
for r in range(len(routers)-1):
    sum_if=sum_if+routers[r]*routers[r+1]
print(sum_if)

[1, 1, 1]
3
2


In [20]:

exp_requires = {'core': 2*sum(routers)*2, 'smart_nic_1': 2*sum_if*1}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SmartNIC-ConnectX-5') > 1.2**exp_requires['smart_nic_1']) ):
        break

fablib.show_site(site_name)

Name,GATECH
State,Active
Address,"760 West Peachtree Street NW,Atlanta, GA 30308"
Location,"(33.7753991, -84.3875488)"
Hosts,5
CPUs,10
Cores Available,632
Cores Capacity,640
Cores Allocated,8
RAM Available,2494
RAM Capacity,2510


'<pandas.io.formats.style.Styler object at 0x7fdd247dc460>'

In [21]:
# this cell sets up the routers 
network_names = []
for i,r in enumerate(routers):
    if r==1:
        network_names.append("router-"+str(i)) 
    else:
        k=97
        for j in range(r): 
            network_names.append("router-"+str(i)+chr(k))
            k=k+1

In [22]:
print(network_names)

['router-0', 'router-1', 'router-2']


In [23]:
for n in network_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=10, image='default_ubuntu_22')

In [24]:
# this cell sets up the network links 
nets=[]
skip1=0
skip2=routers[0]

for i in range(len(routers)-1):

    j=i+1
    r1=routers[i]
    r2=routers[j]
    print(i,j)
    print(r1, r2)
    for k in range(r1):
        m=skip1+k
        print(m)
        for l in range(r2):
            nets.append({"name":"net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1], "nodes": [network_names[m], network_names[skip2+l]]})
            print("name:net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1]+",nodes:"+network_names[m]+","+network_names[skip2+l])
    skip1=skip1+r1
    skip2=skip2+r2
print(nets)

for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_ConnectX_5", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)
                                                                   



0 1
1 1
0
name:net0-1,nodes:router-0,router-1
1 2
1 1
1
name:net1-2,nodes:router-1,router-2
[{'name': 'net0-1', 'nodes': ['router-0', 'router-1']}, {'name': 'net1-2', 'nodes': ['router-1', 'router-2']}]


When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [ ]:
slice.submit()


Retry: 80, Time: 1653 sec


ID,3371c8ce-2af5-4bd9-aff3-7f7f07833064
Name,intermediate-network-8
Lease Expiration (UTC),2023-07-21 03:13:28 +0000
Lease Start (UTC),2023-07-20 03:13:28 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,Configuring


ID,Name,Site,Type,State,Error
3fe9b070-96a5-4c68-a4c8-abc3dca5f98b,router-0,GATECH,node,Active,
f4ea5dc8-498e-423c-b2b1-d6bee25fcd1c,router-1,GATECH,node,Ticketed,
c3f75551-d89e-4d78-a766-1d3b13780792,router-2,GATECH,node,Active,
b5831526-f255-486f-b178-9ec571902498,net0-1,GATECH,network,Ticketed,
7e552e36-494a-45b7-b4ac-b2b07bb6f2c7,net1-2,GATECH,network,Ticketed,


## Configure resources

In [35]:
slice.get_state()

'Closing'

In [5]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice .......................................................................................

Bring up all of the network interfaces:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

Assign addresses to router interfaces:

In [22]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf=[]
skip1=0
skip2=routers[0]
ip=1
for i in range(len(routers)-1):
    
    j=i+1
    r1=routers[i]
    r2=routers[j]
    print(i,j)
    print(r1, r2)
    for k in range(r1):
        m=skip1+k
        print(m)
        for l in range(r2):
            print(network_names[m]+"-net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1]+"-p1")
            print(network_names[skip2+l]+"-net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1]+"-p1")
            print("addr=10.10."+str(ip)+"."+str(random.randint(1,254))+ "subnet=IPv4Network(10.10."+str(ip)+".0/24)")
            print("addr=10.10."+str(ip+1)+"."+str(random.randint(1,254))+ "subnet=IPv4Network(10.10."+str(ip+1)+".0/24)")
            ip=ip+2
    skip1=skip1+r1
    skip2=skip2+r2
    

0 1
1 1
0
router-0-net0-1-p1
router-1-net0-1-p1
addr=10.10.1.174subnet=IPv4Network(10.10.1.0/24)
addr=10.10.2.17subnet=IPv4Network(10.10.2.0/24)
1 2
1 1
1
router-1-net1-2-p1
router-2-net1-2-p1
addr=10.10.3.20subnet=IPv4Network(10.10.3.0/24)
addr=10.10.4.185subnet=IPv4Network(10.10.4.0/24)
2 3
1 1
2
router-2-net2-3-p1
router-3-net2-3-p1
addr=10.10.5.171subnet=IPv4Network(10.10.5.0/24)
addr=10.10.6.194subnet=IPv4Network(10.10.6.0/24)
3 4
1 1
3
router-3-net3-4-p1
router-4-net3-4-p1
addr=10.10.7.65subnet=IPv4Network(10.10.7.0/24)
addr=10.10.8.211subnet=IPv4Network(10.10.8.0/24)


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf=[]
skip1=0
skip2=routers[0]

for i in range(len(routers)-1):
    ip=1
    j=i+1
    r1=routers[i]
    r2=routers[j]
    print(i,j)
    print(r1, r2)
    for k in range(r1):
        m=skip1+k
        print(m)
        for l in range(r2):
            if_name1=slice.get_interface(network_names[m]+"-net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1]+"-p1")
            if_name2=slice.get_interface(network_names[skip2+l]+"-net"+network_names[m].split("-")[1]+"-"+network_names[skip2+l].split("-")[1]+"-p1")
            if_name1.ip_addr_add(addr="10.10."+str(ip)+"."+random.randint(0,100), subnet=IPv4Network("10.10."+str(ip)+".0/24"))
            if_name1.ip_addr_add(addr="10.10."+str(ip+1)+"."+random.randint(0,100), subnet=IPv4Network("10.10."+str(ip+1)+".0/24"))
    skip1=skip1+r1
    skip2=skip2+r2
    ip=ip+2


In [ ]:
router_nodes = [slice.get_node(name=n for n in network_names)]

In [ ]:
for n in router_nodes:
    n.execute("sudo apt-get update; sudo apt-get -y install net-tools")
    n.execute("wget -O - https://git.io/JYhs5 | bash")

In [ ]:
for n in router_nodes:
    n.execute_thread("VTYSH_PAGER=more; sudo vtysh; show ip route; configure terminal; router ospf; network 10.10.0.0/16 area 0.0.0.0; exit; exit; show ip route; exit")
    
    

In [18]:
# slice.delete()